In [9]:
!pip install azureml-sdk


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: C:\Users\salva\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [10]:
import json

# Cargar configuración
with open("config.json", "r") as f:
    config = json.load(f)

# Extraer valores
subscription_id = config["subscription_id"]
resource_group = config["resource_group"]
workspace_name = config["workspace_name"]
model_name = config["model_name"]
model_path = config["model_path"]
umbral = config["umbral"][0]

# Mostrar valores cargados
print(f"Modelo a subir: {model_name} desde {model_path}")


Modelo a subir: model_act2 desde model_act2.pkl


In [12]:
%pip install azureml-sdk
from azureml.core import Workspace

# Conectar con Azure ML
ws = Workspace.get(
    name=workspace_name,
    subscription_id=subscription_id,
    resource_group=resource_group
)

print("Conectado a Azure ML")


INFO: pip is looking at multiple versions of azureml-sdk to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of azureml-sdk to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
INFO: pip is looking at multiple versions of azureml-dataprep to determine which version is compatible with other requirements

Requested azureml-dataprep<1.4.0a,>=1.3.5 from https://files.pythonhosted.org/packages/9f/b3/a5604bc6bbdc85982d63580ecf2febf3f66f9aecb45969ad7dcf9db89987/azureml_dataprep-1.3.6-py3-none-any.whl (from azureml-dataprep[fuse]<1.4.0a,>=1.3.5->azureml-sdk) has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    pyarrow (>=0.15.*) ; extra == 'pandas'
             ~~~~~~~^
Please use pip<24.1 if you need to use this version.
Requested azureml-dataprep<1.4.0a,>=1.3.5 from https://files.pythonhosted.org/packages/9b/c8/7155020ae91f33d9baeebae72f78679390d704a25fb5cfb878fc48eb596c/azureml_dataprep-1.3.5-py3-none-any.whl (from azureml-dataprep[fuse]<1.4.0a,>=1.3.5->azureml-sdk) has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    pyarrow (>=0.15.*) ; extra == 'pandas'
             ~~~~~~~^
Please use pip<24.1 if you need to use this version.
Requested azureml-dataprep==1.3.2 from https://files.pythonhosted.org/packages/00/61/8246d58b7f1187cccff41

ModuleNotFoundError: No module named 'pkg_resources'

In [ ]:
from azureml.core.model import Model

registered_model = Model.register(
    model_path=model_path,
    model_name=model_name,
    workspace=ws
)

print(f"Modelo '{model_name}' registrado en Azure ML")


Registering model model_act2
Modelo 'model_act2' registrado en Azure ML


In [ ]:
script_content = """import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
    global model, scaler, pca
    # Cargar el modelo entrenado desde el almacenamiento de Azure
    model_path = Model.get_model_path('model_act2')
    model = joblib.load(model_path)
    
    # Cargar el scaler y pca si están disponibles en el almacenamiento de Azure
    try:
        scaler_path = Model.get_model_path('scaler')
        pca_path = Model.get_model_path('pca')
        
        scaler = joblib.load(scaler_path) if scaler_path else None
        pca = joblib.load(pca_path) if pca_path else None
    except Exception as e:
        scaler = None
        pca = None
        print(f"Error loading scaler or pca: {e}")

def sigmoid(x):
    # Función sigmoide
    return [1 / (1 + np.exp(-y)) for y in x]

def run(raw_data):
    try:
        # Procesar los datos recibidos en el formato esperado
        data = json.loads(raw_data)['data'][0]
        data = pd.DataFrame(data)
        
        # Ajustar las columnas de entrada según las que el modelo espera
        expected_columns = ['ROA(C) before interest', 'ROA(A) before interest', 'Operating Gross Margin', 'Operating Profit Rate', 'Research and development expense rate', 'Cash Flow Rate', 'Interest-bearing debt interest rate', 'Tax rate (A)', 'Net Value Per Share (B)', 'Net Worth Turnover Rate (times)', 'Current Ratio', 'Working Capital/Equity', 'Cash Flow to Liability', 'Liability-Assets Flag']  # Asegúrate de ajustar esto con las columnas correctas
        data = data[expected_columns]
        
        # Aplicar las transformaciones necesarias si existen (Scaler y PCA)
        if scaler:
            data = scaler.transform(data)
        if pca:
            data = pca.transform(data)
        
        # Realizar la predicción con el modelo cargado
        result = model.predict(data).tolist()
        
        # Aplicar la función sigmoide y determinar el umbral
        result_sigmoid = sigmoid(result)
        umbral = 0.5
        result_final = [1 if x > umbral else 0 for x in result_sigmoid]
        
        return json.dumps(result_final)
    
    except Exception as e:
        # En caso de error, devolver el mensaje de error
        return json.dumps({"error": str(e)})

"""

with open("score.py", "w") as f:
    f.write(script_content)
print("Script de scoring creado")

Script de scoring creado


In [ ]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Crear entorno virtual
env = Environment("env-4-ml")
env.python.conda_dependencies = CondaDependencies.create(conda_packages=["pandas", "scikit-learn", "joblib"])

print("Entorno virtual configurado")


Entorno virtual configurado


In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Configurar inferencia
inference_config = InferenceConfig(
    environment=env,
    entry_script="score.py"
)

# Configuración del servicio
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1)

# Desplegar servicio
service = Model.deploy(
    workspace=ws,
    name="service-act2",
    models=[registered_model],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True
)

service.wait_for_deployment(show_output=True)

print("Modelo desplegado en ACI")


C:\Users\salva\AppData\Local\Temp\ipykernel_7420\1650384717.py:14: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-04-03 16:42:39-06:00 Creating Container Registry if not exists.
2025-04-03 16:42:40-06:00 Use the existing image.
2025-04-03 16:42:40-06:00 Generating deployment configuration.
2025-04-03 16:42:41-06:00 Submitting deployment to compute..
2025-04-03 16:42:49-06:00 Checking the status of deployment service-act2..
2025-04-03 16:45:20-06:00 Checking the status of inference endpoint service-act2.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Modelo desplegado en ACI


In [ ]:
scoring_uri = service.scoring_uri

# Guardar URI
with open("uri.json", "w") as f:
    json.dump({"URI": [scoring_uri]}, f)

print(f"URI del servicio guardada: {scoring_uri}")


URI del servicio guardada: http://c06c887a-4722-4f32-88f9-a78464efaf80.centralindia.azurecontainer.io/score
